In [1]:
from datetime import datetime
from concurrent import futures

import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web

In [1]:
def download_stock(stock):
	""" try to query the iex for a stock, if failed note with print """
	try:
		print(stock)
		stock_df = web.DataReader(stock,'yahoo', start_time, now_time)
		stock_df['Name'] = stock
		output_name = stock + '_data.csv'
		stock_df.to_csv(output_name)
	except:
		bad_names.append(stock)
		print('bad: %s' % (stock))

if __name__ == '__main__':

	""" set the download window """
	now_time = datetime.now()
	start_time = datetime(now_time.year - 5, now_time.month , now_time.day)

	""" list of s_anp_p companies """
	s_and_p = ['^IXIC']
		
	bad_names =[] #to keep track of failed queries

	"""here we use the concurrent.futures module's ThreadPoolExecutor
		to speed up the downloads buy doing them in parallel 
		as opposed to sequentially """

	#set the maximum thread number
	max_workers = 50

	workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
	with futures.ThreadPoolExecutor(workers) as executor:
		res = executor.map(download_stock, s_and_p)

	
	""" Save failed queries to a text file to retry """
	if len(bad_names) > 0:
		with open('failed_queries.txt','w') as outfile:
			for name in bad_names:
				outfile.write(name+'\n')

	#timing:
	finish_time = datetime.now()
	duration = finish_time - now_time
	minutes, seconds = divmod(duration.seconds, 60)
	print('getSandP_threaded.py')
	print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
	#The threaded script took 0 minutes and 31 seconds to run.
df = pd.read_csv('^IXIC_data.csv')
df

^IXIC


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



bad: ^IXIC
Traceback (most recent call last):
  File "C:\Users\rober\anaconda3\envs\time-series\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-838f17d2f364>", line 40, in <module>
    res = executor.map(download_stock, s_and_p)
  File "C:\Users\rober\anaconda3\envs\time-series\lib\concurrent\futures\_base.py", line 636, in __exit__
    self.shutdown(wait=True)
  File "C:\Users\rober\anaconda3\envs\time-series\lib\concurrent\futures\thread.py", line 229, in shutdown
    t.join()
  File "C:\Users\rober\anaconda3\envs\time-series\lib\threading.py", line 1053, in join
    self._wait_for_tstate_lock()
  File "C:\Users\rober\anaconda3\envs\time-series\lib\threading.py", line 1069, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\robe

TypeError: object of type 'NoneType' has no len()

In [2]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [3]:
df=df.drop(columns=['High', 'Low', 'Open', 'Volume', 'Adj Close', 'Name'])

In [4]:
df

,Close
Date,
2016-08-09,5225.479980
2016-08-10,5204.580078
2016-08-11,5228.399902
2016-08-12,5232.890137
2016-08-15,5262.020020
...,...
2021-08-03,14761.290039
2021-08-04,14780.530273
2021-08-05,14895.120117


In [5]:
df['Close'].plot(figsize = (14,5));

In [6]:
IXIC = df

In [7]:
IXIC

,Close
Date,
2016-08-09,5225.479980
2016-08-10,5204.580078
2016-08-11,5228.399902
2016-08-12,5232.890137
2016-08-15,5262.020020
...,...
2021-08-03,14761.290039
2021-08-04,14780.530273
2021-08-05,14895.120117


In [8]:
IXIC.describe()

,Close
count,1259.000000
mean,8441.430810
std,2581.905875
min,5046.370117
25%,6632.164795
50%,7746.379883
75%,9440.165039
max,14895.120117


In [9]:
cutoff = round(IXIC.shape[0]*0.9)

In [10]:
cutoff

1133

In [11]:
from random import gauss as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import itertools
#from pmdarima import auto_arima

#statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [12]:
train = IXIC[:cutoff]
test = IXIC[cutoff:]

fig, ax = plt.subplots()
ax.plot(train)
ax.plot(test)
ax.set_title('Train test Split');

In [13]:
def download_stock(stock):
	""" try to query the iex for a stock, if failed note with print """
	try:
		print(stock)
		stock_df = web.DataReader(stock,'yahoo', start_time, now_time)
		stock_df['Name'] = stock
		output_name = stock + '_data.csv'
		stock_df.to_csv(output_name)
	except:
		bad_names.append(stock)
		print('bad: %s' % (stock))

if __name__ == '__main__':

	""" set the download window """
	now_time = datetime.now()
	start_time = datetime(now_time.year - 5, now_time.month , now_time.day)

	""" list of s_anp_p companies """
	s_and_p = ['^GSPC']
		
	bad_names =[] #to keep track of failed queries

	"""here we use the concurrent.futures module's ThreadPoolExecutor
		to speed up the downloads buy doing them in parallel 
		as opposed to sequentially """

	#set the maximum thread number
	max_workers = 50

	workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
	with futures.ThreadPoolExecutor(workers) as executor:
		res = executor.map(download_stock, s_and_p)

	
	""" Save failed queries to a text file to retry """
	if len(bad_names) > 0:
		with open('failed_queries.txt','w') as outfile:
			for name in bad_names:
				outfile.write(name+'\n')

	#timing:
	finish_time = datetime.now()
	duration = finish_time - now_time
	minutes, seconds = divmod(duration.seconds, 60)
	print('getSandP_threaded.py')
	print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
	#The threaded script took 0 minutes and 31 seconds to run.
GSPC = pd.read_csv('^GSPC_data.csv')
GSPC

^GSPC
bad: ^GSPC
getSandP_threaded.py
The threaded script took 0 minutes and 3 seconds to run.


,Date,High,Low,Open,Close,Volume,Adj Close,Name
0,2016-08-09,2187.659912,2178.610107,2182.239990,2181.739990,3334300000,2181.739990,^GSPC
1,2016-08-10,2183.409912,2172.000000,2182.810059,2175.489990,3254950000,2175.489990,^GSPC
2,2016-08-11,2188.449951,2177.969971,2177.969971,2185.790039,3423160000,2185.790039,^GSPC
3,2016-08-12,2186.280029,2179.419922,2183.739990,2184.050049,3000660000,2184.050049,^GSPC
4,2016-08-15,2193.810059,2186.080078,2186.080078,2190.149902,3078530000,2190.149902,^GSPC
...,...,...,...,...,...,...,...,...
1254,2021-08-03,4423.790039,4373.000000,4392.740234,4423.149902,3305340000,4423.149902,^GSPC
1255,2021-08-04,4416.169922,4400.229980,4415.950195,4402.660156,3382620000,4402.660156,^GSPC
1256,2021-08-05,4429.759766,4408.859863,4408.859863,4429.100098,2734220000,4429.100098,^GSPC
1257,2021-08-06,4440.819824,4429.069824,4429.069824,4436.520020,2839970000,4436.520020,^GSPC


In [14]:
from sklearn.pipeline import Pipeline

In [15]:
def clean(df, train_size):
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df=df.drop(columns=['High', 'Low', 'Open', 'Volume', 'Adj Close', 'Name'])
    cutoff = round(df.shape[0]*train_size)
    train = df[:cutoff]
    test = df[cutoff:]
    return train, test

In [16]:
def download_stock(stock):
	""" try to query the iex for a stock, if failed note with print """
	try:
		print(stock)
		stock_df = web.DataReader(stock,'yahoo', start_time, now_time)
		stock_df['Name'] = stock
		output_name = stock + '_data.csv'
		stock_df.to_csv(output_name)
	except:
		bad_names.append(stock)
		print('bad: %s' % (stock))

if __name__ == '__main__':

	""" set the download window """
	now_time = datetime.now()
	start_time = datetime(now_time.year - 5, now_time.month , now_time.day)

	""" list of s_anp_p companies """
	s_and_p = ['^DJI']
		
	bad_names =[] #to keep track of failed queries

	"""here we use the concurrent.futures module's ThreadPoolExecutor
		to speed up the downloads buy doing them in parallel 
		as opposed to sequentially """

	#set the maximum thread number
	max_workers = 50

	workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
	with futures.ThreadPoolExecutor(workers) as executor:
		res = executor.map(download_stock, s_and_p)

	
	""" Save failed queries to a text file to retry """
	if len(bad_names) > 0:
		with open('failed_queries.txt','w') as outfile:
			for name in bad_names:
				outfile.write(name+'\n')

	#timing:
	finish_time = datetime.now()
	duration = finish_time - now_time
	minutes, seconds = divmod(duration.seconds, 60)
	print('getSandP_threaded.py')
	print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
	#The threaded script took 0 minutes and 31 seconds to run.


^DJI
bad: ^DJI
getSandP_threaded.py
The threaded script took 0 minutes and 3 seconds to run.


In [3]:
DJI = pd.read_csv('^DJI_data.csv')


In [4]:
DJI_train, DJI_test = clean(DJI, .9)

NameError: name 'clean' is not defined

In [ ]:
DJI_train

In [20]:
fig, ax = plt.subplots()
ax.plot(DJI_train)
ax.plot(DJI_test)
ax.set_title('Train test Split');

In [21]:
#!pip install --upgrade tensorflow
#import tensorflow as tf 
#from tensorflow import keras
#from tensorflow.keras import models
#from tensorflow.keras import layers
#from tensorflow.keras import optimizers
#from sklearn.preprocessing import MinMaxScaler
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM



#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:987,:]
valid = dataset[987:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [22]:
def download_stock(stock):
	""" try to query the iex for a stock, if failed note with print """
	try:
		print(stock)
		stock_df = web.DataReader(stock,'yahoo', start_time, now_time)
		stock_df['Name'] = stock
		output_name = stock + '_data.csv'
		stock_df.to_csv(output_name)
	except:
		bad_names.append(stock)
		print('bad: %s' % (stock))

if __name__ == '__main__':

	""" set the download window """
	now_time = datetime.now()
	start_time = datetime(now_time.year - 5, now_time.month , now_time.day)

	""" list of s_anp_p companies """
	s_and_p = ['GC=F']
		
	bad_names =[] #to keep track of failed queries

	"""here we use the concurrent.futures module's ThreadPoolExecutor
		to speed up the downloads buy doing them in parallel 
		as opposed to sequentially """

	#set the maximum thread number
	max_workers = 50

	workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
	with futures.ThreadPoolExecutor(workers) as executor:
		res = executor.map(download_stock, s_and_p)

	
	""" Save failed queries to a text file to retry """
	if len(bad_names) > 0:
		with open('failed_queries.txt','w') as outfile:
			for name in bad_names:
				outfile.write(name+'\n')

	#timing:
	finish_time = datetime.now()
	duration = finish_time - now_time
	minutes, seconds = divmod(duration.seconds, 60)
	print('getSandP_threaded.py')
	print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
	#The threaded script took 0 minutes and 31 seconds to run.


GC=F
bad: GC=F
getSandP_threaded.py
The threaded script took 0 minutes and 3 seconds to run.


In [2]:
Gold = pd.read_csv('GC=F_data.csv')


In [3]:
from prophet import Prophet as proph

In [4]:
Model = proph()

In [5]:
Gold.reset_index(inplace=True)

In [6]:
Gold['ds']=Gold['Date']

In [7]:
Gold['y']=Gold['Close']

In [8]:
Gold.drop(columns=['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'Name',], inplace=True)

In [9]:
cutoff=round(Gold.shape[0]*0.9)

In [10]:
Gold_train = Gold[:cutoff]
Gold_test = Gold[cutoff:]

In [11]:
Model.fit(Gold_train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


AttributeError: 'StanModel' object has no attribute 'fit_class'

In [ ]:
Gold

In [ ]:
def download_stock(stock):
	""" try to query the iex for a stock, if failed note with print """
	try:
		print(stock)
		stock_df = web.DataReader(stock,'yahoo', start_time, now_time)
		stock_df['Name'] = stock
		output_name = stock + '_data.csv'
		stock_df.to_csv(output_name)
	except:
		bad_names.append(stock)
		print('bad: %s' % (stock))

if __name__ == '__main__':

	""" set the download window """
	now_time = datetime.now()
	start_time = datetime(now_time.year - 5, now_time.month , now_time.day)

	""" list of s_anp_p companies """
	s_and_p = ['^RUT']
		
	bad_names =[] #to keep track of failed queries

	"""here we use the concurrent.futures module's ThreadPoolExecutor
		to speed up the downloads buy doing them in parallel 
		as opposed to sequentially """

	#set the maximum thread number
	max_workers = 50

	workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
	with futures.ThreadPoolExecutor(workers) as executor:
		res = executor.map(download_stock, s_and_p)

	
	""" Save failed queries to a text file to retry """
	if len(bad_names) > 0:
		with open('failed_queries.txt','w') as outfile:
			for name in bad_names:
				outfile.write(name+'\n')

	#timing:
	finish_time = datetime.now()
	duration = finish_time - now_time
	minutes, seconds = divmod(duration.seconds, 60)
	print('getSandP_threaded.py')
	print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')